In [2]:
pip install imblearn

In [3]:
import pandas as pd
import numpy as np
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
stop = stopwords.words('russian')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import Counter
from imblearn.datasets import fetch_datasets
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [4]:
def print_results(headline, true_value, pred):
    print(headline)
    print("accuracy: {}".format(accuracy_score(true_value, pred)))
    print("precision: {}".format(precision_score(true_value, pred)))
    print("recall: {}".format(recall_score(true_value, pred)))
    print("f1: {}".format(f1_score(true_value, pred)))

In [5]:
df =  pd.read_csv(r"C:\Users\vladr\NLP_Project_SchoolX-main\dataset\data.csv", names = ['Review', 'Rating'], sep=',')
df = df.dropna(how='any',axis=0)
np.random.seed(37)
df = df.drop_duplicates(subset=['Review', 'Rating'], keep=False)
df = df.reset_index(drop=True)

In [6]:
df['Rating']=df['Rating'].astype(int)
df=df[df['Rating']!=3]
df['label']=np.where(df['Rating']>=3,1,0)

In [7]:
df['Rating'].value_counts()

5    107885
4     50785
2     16121
1     15511
Name: Rating, dtype: int64

In [8]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle
data=df[df['label']==0][:30000]
data=data.append(df[df['label']==1][:150000])
data = data.reset_index(drop=True)
display(data['label'].value_counts())
data

1    150000
0     30000
Name: label, dtype: int64

,Review,Rating,label
0,"Жалко деньги, что были потрачены на такой отвр...",1,0
1,"Меньше, чем за 1 год 4 раза отклеивалась перед...",1,0
2,После или в сравнении с андроидом - небо и зем...,1,0
3,"Покупать не советую, за такую цену сырая модель.",2,0
4,Отзыв написан после 8-ми месячной эксплуатации...,2,0
...,...,...,...
179995,"Качественный бюджетный телефон. Умеет все, что...",5,1
179996,Лично у меня к нему отношение как к хорошему и...,5,1
179997,Ну что вы хотите? За такие деньги получить суп...,4,1
179998,"не смотря на падения жив-здоров, только в неко...",5,1


In [9]:
data['preprocess'] = data['Review'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['preprocess']=data['preprocess'].apply(lambda x: BeautifulSoup(x).get_text())
data['preprocess']=data['preprocess'].apply(lambda x: re.sub(r"http\S+", "", x))
data['preprocess']=data['preprocess'].apply(lambda x: "".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))
data['preprocess']=data['preprocess'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

D:\anaconda\lib\site-packages\bs4\__init__.py:417: MarkupResemblesLocatorWarning: "http://www.youtube.com/watch?v=o6gonrvww2y&feature=youtu.be" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
D:\anaconda\lib\site-packages\bs4\__init__.py:417: MarkupResemblesLocatorWarning: "http://www.youtube.com/watch?v=szriytnwbfc&list=uutj4atedkibllwarijz6bnq&index=1&feature=plcp" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
D:\anaconda\lib\site-packages\bs4\__init__.py:332: MarkupResemblesLocatorWarning: "...." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
D:\anaconda\lib\site-packages\bs4\__init__.py:332

In [10]:
X_train,X_test,Y_train, Y_test = train_test_split(data['preprocess'], data['label'], test_size=0.25, random_state=30)
print("Train:",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train: (135000,) (135000,) Test:  ((45000,), (45000,))


In [11]:
vectorizer = TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

In [45]:
#classifier = LinearSVC(random_state=58, max_iter=100)
classifier = RandomForestClassifier

In [47]:
# build normal model with vectorizer
vectorized_normal_pipeline = make_pipeline(classifier(random_state=42))
vectorized_normal_model = pipeline.fit(tf_x_train, Y_train)
vectorized_normal_prediction = model.predict(tf_x_test)

AttributeError: 'function' object has no attribute 'fit'

In [19]:
# build model with SMOTE imblearn
smote_pipeline = make_pipeline_imb(SMOTE(random_state=4), classifier(random_state=42))
smote_model = smote_pipeline.fit(tf_x_train, Y_train)
smote_prediction = smote_model.predict(tf_x_test)

In [25]:
from sklearn.linear_model import LogisticRegression

In [27]:
# build model with undersampling
nr = NearMiss()
X_train_miss, y_train_miss = nr.fit_sample(tf_x_train,Y_train.ravel())
lr2 = LogisticRegression()
lr2.fit(X_train_miss, y_train_miss.ravel())
nearmiss_prediction = lr2.predict(tf_x_test)

AttributeError: 'NearMiss' object has no attribute 'fit_sample'

In [44]:
# print information about both models
print("normal data distribution: {}".format(Counter(data['label'])))
X_smote, y_smote = SMOTE().fit_resample(data['preprocess'], data['label']) #ругается на то что тут буквавки а не циферки, надо заранее векторизировать
print("SMOTE data distribution: {}".format(Counter(y_smote)))
#X_nearmiss, y_nearmiss = NearMiss().fit_sample(data['data'], data['target'])
#print("NearMiss data distribution: {}".format(Counter(y_nearmiss)))

normal data distribution: Counter({1: 150000, 0: 30000})


ValueError: Found input variables with inconsistent numbers of samples: [135000, 180000]

In [32]:
# classification report
print(classification_report(Y_test, vectorized_normal_prediction))
print(classification_report_imbalanced(Y_test, smote_prediction))

              precision    recall  f1-score   support

           0       0.81      0.13      0.22      7470
           1       0.85      0.99      0.92     37530

    accuracy                           0.85     45000
   macro avg       0.83      0.56      0.57     45000
weighted avg       0.84      0.85      0.80     45000

                   pre       rec       spe        f1       geo       iba       sup

          0       0.20      0.93      0.25      0.33      0.48      0.24      7470
          1       0.95      0.25      0.93      0.39      0.48      0.21     37530

avg / total       0.82      0.36      0.82      0.38      0.48      0.22     45000



In [37]:

print('vectorized_normal Pipline Score{}'.format(vectorized_normal_pipeline.score(tf_x_test, Y_test)))
print('SMOTE Pipeline Score {}'.format(smote_pipeline.score(tf_x_test, Y_test)))
#print('NearMiss Pipeline Score {}'.format(nearmiss_pipeline.score(tf_x_test, Y_test)))

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [39]:

print_results("vectorized normal classification", Y_test, vectorized_normal_prediction)

print_results("SMOTE classification", Y_test, smote_prediction)

print_results("NearMiss classification", Y_test, nearmiss_prediction)

vectorized normal classification
accuracy: 0.8500666666666666
precision: 0.8510114255741293
recall: 0.9942978950173195
f1: 0.9170916329765664
SMOTE classification
accuracy: 0.35964444444444443
precision: 0.9466885380356777
recall: 0.2460431654676259
f1: 0.3905760933931139


In [43]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test, smote_prediction)


0.588215692506236